In [ ]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
import re

# 测试每个分子pdb文件是否能被正确识别

In [ ]:
pdb = PDBFile('EC.pdb')
forcefield = ForceField('EC_Amoeba.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)

In [ ]:
pdb = PDBFile('DMC.pdb')
forcefield = ForceField('DMC_Amoeba.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)

In [ ]:
pdb = PDBFile('PF6n1.pdb')
forcefield = ForceField('PF6n1_Amoeba.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)

In [ ]:
pdb = PDBFile('Lip1.pdb')
forcefield = ForceField('Lip1_Amoeba.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)

# 生成包含若干LiPF6、EC和DMC的pdb

In [ ]:
#define function
def Get_TER_END_lines(file_path: str, start_str: str = "TER", end_str: str = "END") -> str:
    """
    从文件中获取起始字符串和终止字符串之间的行。

    参数：
    file_path (str): 包含需要查找行的文件路径。
    start_str (str): 起始字符串。默认为"TER"。
    end_str (str): 终止字符串。默认为"END"。

    返回值：
    str: 包含起始字符串和终止字符串之间行的字符串，如果未找到，则返回空字符串。
    """
    # 从文件中读取文本
    with open(file_path, "r") as f:
        content = f.read()

    # 使用正则表达式匹配起始字符串和终止字符串之间的行
    pattern = fr"{start_str}.*\n(.*\n)*?{end_str}"
    match = re.search(pattern, content)

    if match:
        # 返回匹配到的字符串,用切片剔除第一行和最后一行
        lines = (match.group().splitlines()[1:-1])
        return '\n'.join(lines)
    else:
        # 打印错误信息
        print("没有匹配到起始字符串和终止字符串之间的内容。")
        return ""

def Add_CONECT(CONECT_txt: str, nmol: int, last_atom_index: int) -> str:
    """
    将传入的CONECT_txt字符串解析为原子索引的列表，然后根据给定的nmol和last_atom_index参数来生成新的CONECT行，
    并将它们存储在一个新的字符串列表newlines中，最后将其作为返回值返回。
    
    参数:
    CONECT_txt(str): 包含CONECT行的字符串。
    nmol(int): 指定需要生成多少个分子的CONECT行。
    last_atom_index(int): 上一个分子最后的原子index。

    返回值:
    newlines(str): 一个包含新的CONECT行的str。
    """
    # 从connect_txt字符串中解析出原子索引的列表。
    atom_indices = re.findall(r"\d+", CONECT_txt)

    # 计算出原子索引列表中的最大值，即该分子的原子数
    max_index = max([int(num) for num in atom_indices])

    # 将connect_txt字符串按行拆分，并遍历每一行。
    origin_lines = CONECT_txt.split("\n")
    newlines = []
    for mol in range(nmol):
        for connect in origin_lines:
            # 根据传入的参数，生成新的CONECT行。
            atom_indices = re.findall(r"\d+", connect)
            new_index1 = int(atom_indices[0]) + last_atom_index + mol * max_index
            new_index2 = int(atom_indices[1]) + last_atom_index + mol * max_index
            newline = "CONECT{:>5}{:>5}".format(new_index1, new_index2)
            newlines.append(newline)

    # 将newlines作为返回值返回。
    return "\n".join(newlines)+"\n"

def Add_Newline(file_path:str,new_lines:str,before_which_string:str) -> None:
    """
    在指定字符串之前插入新的文本行到一个文件中。

    参数：
        filename (str): 要修改的文件名
        new_lines (list): 要插入的文本
        before_which_string (str): 在该字符串所在行的上一行插入新行

    返回：
        无返回值，但会将修改后的内容写回文件中。

    """
    # 打开文件并读取所有内容为列表
    with open(file_path, "r") as file:
        content = file.readlines()

    # 在文件中查找“before_which_string”所在行
    for index, line in enumerate(content):
        if before_which_string in line:
            break

    # 在找到的“before_which_string”行前两行的位置插入新行
    content[index:index] = new_lines

    # 将修改后的内容写回文件
    with open(file_path, "w") as file:
        file.writelines(content)

注意，packmol在windows和Linux上的表现略有不同，但是都不能正确地生成键（连接）信息，需要自己额外添加。
盒子的大小原则上可由密度计算确定，实际上由于npt会自动调整盒子大小，因此差不多合适即可，容易通过二分法大致估计。
如果packmol运行很快，则说明盒子过大，填充过于容易，如果运行花了若干秒，则基本上可以确定合适。

In [ ]:
#写入packmol需要的.inp文件
inp = """#
# mixture LiPF6, EC and DMC
#

tolerance 2.0
filetype pdb
add_box_sides 1

output LiPF6_EC_DMC.pdb

structure Lip1.pdb
  number 49
  inside box 0. 0. 0. 40. 40. 40. 
end structure

structure PF6n1.pdb
  number 49
  inside box 0. 0. 0. 40. 40. 40. 
end structure

structure EC.pdb
  number 160
  inside box 0. 0. 0. 40. 40. 40. 
end structure

structure DMC.pdb
  number 360
  inside box 0. 0. 0. 40. 40. 40. 
end structure"""

with open('mixture.inp','w') as w:
    w.write(inp)

In [ ]:
!packmol < mixture.inp

In [ ]:
#依次增加PF6-，EC和DMC的成键信息到pdb文件中
CONECT_txt = Get_TER_END_lines('PF6n1.pdb')
nmol = 49
first_atom_index = 49
new_lines = Add_CONECT(CONECT_txt, nmol, first_atom_index)
Add_Newline('LiPF6_EC_DMC.pdb', new_lines, 'END')

CONECT_txt = Get_TER_END_lines('EC.pdb')
nmol = 160
first_atom_index += 49 * 7
new_lines = Add_CONECT(CONECT_txt, nmol, first_atom_index)
Add_Newline('LiPF6_EC_DMC.pdb', new_lines, 'END')

CONECT_txt = Get_TER_END_lines('DMC.pdb')
nmol = 480
first_atom_index += 160 * 10
new_lines = Add_CONECT(CONECT_txt, nmol, first_atom_index)
Add_Newline('LiPF6_EC_DMC.pdb', new_lines, 'END')

# npt

为了得到合适的密度，先使用npt模拟。由于AMOEBA是一个更“精细”的力场，对原子细节有描述，这里时间步长应该取小一点较为合适，2fs略长。步长过长也会导致收敛性差，很容易跑崩。

In [ ]:
pdb = PDBFile('LiPF6_EC_DMC.pdb')
forcefield = ForceField('Lip1_Amoeba.xml','PF6n1_Amoeba.xml','DMC_Amoeba.xml','EC_Amoeba.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, vdwCutoff=1.2*nanometer, polarization='extrapolated')

platform = Platform.getPlatformByName('CUDA')
properties = {'DeviceIndex': '0', 'Precision': 'single'}
integrator = LangevinMiddleIntegrator(333*kelvin, 1/picosecond, 0.002*picoseconds)
system.addForce(MonteCarloBarostat(1*bar, 333*kelvin))
simulation = Simulation(pdb.topology, system, integrator, platform,properties)

In [ ]:
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()
simulation.reporters.append(
    DCDReporter('npt10ns.dcd', 1000, enforcePeriodicBox=True))
simulation.reporters.append(
    StateDataReporter('data.csv',
                      1000,
                      time=True,
                      totalEnergy=True,
                      volume=True,
                      temperature=True))
simulation.reporters.append(
    StateDataReporter(stdout,
                      100,
                      step=True,
                      speed=True,
                      volume=True,
                      totalEnergy=True,
                      temperature=True))

In [ ]:
simulation.step(5000000)

# nvt

由于上一步忘了保存最后一帧的结构和速度，因此使用MDAnalysis重新拿到最后一帧的坐标以及盒子大小信息。

In [ ]:
import MDAnalysis as mda

u = mda.Universe('LiPF6_EC_DMC.pdb','npt10ns.dcd')

last_frame = u.trajectory[-1]
with mda.Writer('nvt_last_frame.pdb', u.atoms.n_atoms) as writer:
    writer.write(u.atoms)

进行nvt模型，步长设为1fs（背后的原因是一开始设置2fs老是跑一会就爆炸，后来才想起来是步长的问题，令人暖心）

In [ ]:
pdb = PDBFile('nvt_last_frame.pdb')
forcefield = ForceField('Lip1_Amoeba.xml','PF6n1_Amoeba.xml','DMC_Amoeba.xml','EC_Amoeba.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, vdwCutoff=1.2*nanometer, polarization='extrapolated')

platform = Platform.getPlatformByName('CUDA')
properties = {'DeviceIndex': '0', 'Precision': 'single'}
integrator = LangevinMiddleIntegrator(333*kelvin, 1/picosecond, 0.001*picoseconds)
simulation = Simulation(pdb.topology, system, integrator, platform,properties)
simulation.context.setPositions(pdb.positions)
simulation.context.setVelocitiesToTemperature(333*kelvin)

In [ ]:
# simulation.minimizeEnergy()
simulation.reporters.append(
    DCDReporter('nvt10ns.dcd', 1000, enforcePeriodicBox=True))
simulation.reporters.append(
    StateDataReporter('data.csv',
                      1000,
                      time=True,
                      totalEnergy=True,
                      temperature=True))
simulation.reporters.append(
    StateDataReporter(stdout,
                      1000,
                      step=True,
                      speed=True,
                      density=True,
                      totalEnergy=True,
                      temperature=True))

In [ ]:
simulation.step(10000000)

In [ ]:
simulation.saveState('nvt10ns.xml')

In [ ]:
simulation.saveCheckpoint('nvt10ns.chk')